In [27]:
import pandas as pd
import numpy as np
import heapq
import queue

In [28]:
TIME_TOTAL = 20 # in the unit of month
N_REVIEWERS = 30
N_ARTICLES_PER_ISSUE = 10
ACCEPT_RATE_EDITOR = .1
ACCEPT_RATE_REVIEWER = .9

The idea of the algorithm is as follows:
* Event is the trigger.
* Paper is the processed.
* Reviewer is the processor.

In [29]:
class Event:
    """
    Event mimics the behavior of the timeline. Each time an event occurs, corresponding objects change accordingly.
    The operators "<", "<=", "==", ">=", ">" are overridden so that the event container -- Min-Heap, can sort its events.
    ---------------------------------------------------------------------------------------------------------------------
    kind: There are three kind of events: "paper_in", "paper_out" and "issue".
          "paper_in" is an incoming paper, whose time_occur is predetermined and whose quantity per month is subject to N(110, 10^2)
          "paper_out" is an outcoming reviewed paper.
          "issue" is an issuance of journal, whose time_occur is predetermined.
    time_occur: time when this event occurs.
    """
    
    def __init__(self, kind, time_occur):
        self.kind = kind
        self.time_occur = time_occur
    
    def __lt__(self, other):
        return self.time_occur < other.time_occur
    
    def __gt__(self, other):
        return self.time_occur > other.time_occur
    
    def __eq__(self, other):
        return self.time_occur == other.time_occur
    
    def __le__(self, other):
        return self.time_occur <= other.time_occur
    
    def __ge__(self, other):
        return self.time_occur >= other.time_occur

In [30]:
class Paper:
    """
    Paper is the target for analysis. Its attributes change during the whole process.
    ---------------------------------------------------------------------------------
    accepted_by_editor: if this paper is accepted by editors or not
    accepted_by_reviewer: if this paper is accepted by reviewers or not
    time_in: time when the review of this paper is started
    time_out: time when the review of this paper is finished
    time_published: time when the paper is published
    """
    
    def __init__(self, accepted_by_editor, accepted_by_reviewer, time_in, time_out, time_published):
        self.accepted_by_editor = accepted_by_editor
        self.accepted_by_reviewer = accepted_by_reviewer
        self.time_in = time_in
        self.time_out = time_out
        self.time_published = time_published

In [31]:
class Reviewer:
    """
    Reviewer is the processor. 
    It pushes a Paper in its queue when an event "paper_in" occurs. It pops a Paper out of its queue when an event "paper_out" occur.
    The operators "<", "<=", "==", ">=", ">" are overridden so that the event container -- Min-Heap, can sort its events,
    so that when an event "paper_out" occurs, the Reviewer of this paper is at the exit of the Min-Heap.
    ---------------------------------------------------------------------------------------------------------------------------------
    time_next: time when the reviewer finishes reviewing the paper on hand and starts reviewing the next paper, 
               only for deciding which reviewer to choose when assigning a new paper.
    n_paper: num of papers the reviewer has, including the one under review.
    paper_processing_qu: a Queue (First-in-First-out) which contains all the Papers the reviewer has.
    ACCEPT_RATE_REVIEWER: the probability that the reviewer accepts a Paper.
    """
    
    def __init__(self, ACCEPT_RATE_REVIEWER, paper_processing_qu, time_next = 0, n_paper = 0):
        self.time_next = time_next
        self.n_paper = n_paper
        self.paper_processing_qu = paper_processing_qu
        self.ACCEPT_RATE_REVIEWER  = ACCEPT_RATE_REVIEWER
        
    def accept(self, event, paper):
        """
        Accept a new paper. (REQUIRE HEAPIFY)
        """
        import numpy as np
        
        if event.kind != "paper_in":  # Check the correctness of event kind.
            raise Exception("Event is not paper_in!")
        
        self.paper_processing_qu.put(paper) # Add this Paper to the Paper Queue.
        self.n_paper = self.n_paper + 1 # Add the num of total papers by 1.
        event_finish = self.start(event.time_occur)
        return event_finish
    
    def finish(self, event):
        """
        Finish an old paper. (REQUIRE HEAPIFY)
        """
        import numpy as np
        
        if event.kind != "paper_out":  # Check the correctness of event kind.
            raise Exception("Event is not paper_out!")
        
        paper_old = self.paper_processing_qu.get(False) # Pop the old paper out of the Paper Queue.
        self.n_paper = self.n_paper - 1 # Add the num of total papers by 1.
        
        
        if event.time_occur != paper_old.time_out:
            raise Exception("Paper is under review!")
        
        accept_or_not = np.random.binomial(n = 1, p = self.ACCEPT_RATE_REVIEWER) == 1
        paper_old.accepted_by_reviewer = accept_or_not
        
        event_finish = self.start(event.time_occur)
        
        return paper_old, event_finish
    
    def start(self, time_current):
        """
        Start to review a new paper. This method is called within the class.
        """
        event_finish = None
        
        if not self.paper_processing_qu.empty(): # if the Paper Queue is not empty
            paper_new = self.paper_processing_qu.queue[0] # Get but not pop the first Paper.
            if paper_new.time_out is None: # if the new paper is not under review
                time_duration = np.random.normal(loc = 6, scale = 1) # the time needed for this review
                time_finish = time_current + time_duration # the time when the review if finished
                paper_new.time_out = time_finish # set the time_out attribute of the Paper
                self.compute_time_next(time_current)
                event_finish = Event(kind = "paper_out", time_occur = time_finish)
                
        return event_finish # return None if there is no more papers to review or the new paper is already under review. Otherwise, return a "paper_out" event with the finishing time.
    
    def compute_time_next(self, time_current):
        """
        Adjust time_next. This method is called either when a new paper comes in or an old paper comes out.
        """
        if self.paper_processing_qu.empty(): # if there is no more papers to review, which means the method is called by the function finish
            self.time_next = time_current # set time_next to be the time when the reviewer is free (now).
        else:
            self.time_next = self.paper_processing_qu.queue[0].time_out
    
    def __lt__(self, other):
        return self.time_next < other.time_next
    
    def __gt__(self, other):
        return self.time_next > other.time_next
    
    def __eq__(self, other):
        return self.time_next == other.time_next
    
    def __le__(self, other):
        return self.time_next <= other.time_next
    
    def __ge__(self, other):
        return self.time_next >= other.time_next
        
    def __lshift__(self, other):
        return self.n_paper < other.n_paper or (self.n_paper == other.n_paper and self.time_next < other.time_next)

In [32]:
def get_most_free_reviewer(reviewer_hp):
    most_free_reviewer = reviewer_hp[0]
    for reviewer in reviewer_hp:
        if reviewer << most_free_reviewer:
            most_free_reviewer = reviewer
    
    return most_free_reviewer

In [33]:
time_current = 0
event_hp = []
reviewer_hp = []
paper_reviewed_qu = queue.Queue()
paper_published_qu = queue.Queue()
paper_rejected_by_editor_qu = queue.Queue()
paper_rejected_by_reviewer_qu = queue.Queue()

In [34]:
# Initialization
for month in range(TIME_TOTAL):
    heapq.heappush(event_hp, Event(kind = "issue", time_occur = month))
    n_paper_submitted = int(np.random.normal(loc = 110, scale = 10))
    for i_paper in range(n_paper_submitted):
        heapq.heappush(event_hp, Event(kind = "paper_in", time_occur = month))

for i_reviewer in range(N_REVIEWERS):
    heapq.heappush(reviewer_hp, Reviewer(ACCEPT_RATE_REVIEWER = ACCEPT_RATE_REVIEWER, paper_processing_qu = queue.Queue()))

In [35]:
while len(event_hp) > 0:
    event = heapq.heappop(event_hp)
    time_current = event.time_occur
    if event.kind == "paper_in":
        accepted_by_editor = np.random.binomial(n = 1, p = ACCEPT_RATE_EDITOR) == 1
        paper_new = Paper(accepted_by_editor = accepted_by_editor,
                          accepted_by_reviewer = None,
                          time_in = time_current,
                          time_out = None,
                          time_published = None)
        if not paper_new.accepted_by_editor:
            paper_rejected_by_editor_qu.put(paper_new)
            continue
        most_free_reviewer = get_most_free_reviewer(reviewer_hp)
        event_finish = most_free_reviewer.accept(event, paper_new)
        if event_finish is not None:
            heapq.heappush(event_hp, event_finish)
            
    elif event.kind == "paper_out":
        heapq.heapify(reviewer_hp)
        for i_reviewer in range(N_REVIEWERS): # The reviewer for this paper is mostly likely to be at the exit. Unless there are free reviewers.
            reviewer_current = reviewer_hp[i_reviewer]
            if reviewer_current.time_next == time_current:
                paper_old, event_finish = reviewer_current.finish(event)
                break
                
        if event_finish is not None:
            heapq.heappush(event_hp, event_finish)
        if paper_old.accepted_by_reviewer:
            paper_reviewed_qu.put(paper_old)
        else:
            paper_rejected_by_reviewer_qu.put(paper_old)
            
    else:
        n_paper = 0
        while n_paper < N_ARTICLES_PER_ISSUE and not paper_reviewed_qu.empty():
            paper_published = paper_reviewed_qu.get(False)
            n_paper += 1
            paper_published.time_published = time_current
            paper_published_qu.put(paper_published)

In [36]:
paper_reviewed_qu.qsize()

101

In [37]:
paper_published_qu.qsize()

67

In [38]:
paper_rejected_by_editor_qu.qsize()

2008

In [39]:
paper_rejected_by_reviewer_qu.qsize()

24